## Recommendation Systems
В качестве тестового задания вам предлагается проанализировать посты из ленты «ВКонтакте» и подготовить модели для предсказания конверсии из просмотра поста в различные активности, например, лайк или пересылка в личные сообщения.

Датасет доступен по ссылке, это csv-таблица, где колонки text и photo содержат текст и визуал поста, а view отвечает за число просмотра. Остальные колонки содержат информацию об активностях: число лайков, число комментариев, число, когда пост скрывали или открывали, когда открывали пост отдельно и когда его пересылали в личных сообщениях.

Изображения хранятся в сжатом виде, для их просмотра можно воспользоваться, например, следующим фрагментом кода:

    from base64 import b64decode
    from io import BytesIO
    from PIL import Image

    img = Image.open(BytesIO(b64decode(data.loc[img_index, "photo"])))

### Задачи:

1. Скачайте датасет и подготовьте его к работе. Проведите первичный разведывательный анализ, определитесь с активностями, с которыми будете работать. Подготовьте датасет к обучению, сформируйте нужные выборки и целевые переменные.
Важно: в качестве задания вам надо предсказать конверсию просмотра в активность, т.е. ее вероятность. В качестве метрики рекомендуем ориентироваться на правдоподобие, но вы можете добавить свои.
2. Эксперименты! Вы можете использовать как текст, так и изображения. Все вместе или по отдельности. Добейтесь как можно более высокого качества используя доступные open-source модели и обучая свои.
3. Проанализируйте и сделайте выводы по полученным моделям. Сформируйте полезные советы для авторов, которые можно использовать для повышения конверсии.

Оформите задание в виде репозитория на GitHub, мы будем отдельно проверять и глубину погружения в эксперименты, и качество кода. В качестве решения пришлите ссылку на репозиторий.

## Ответ на задание

1. Подготовка данных и выбор активностей:
		
		Я загрузил датасет и провёл разведывательный анализ. Выбрал следующие активности для предсказания конверсии: лайки, комментарии, скрытие, разворачивание, открытие фотографии, пересылка и прочие. Эти активности были объединены в одну метрику с помощью весов для каждой активности, а целевая переменная представляла собой отношение суммы активностей к числу просмотров.

2. Целевая переменная:
		
		Целевая переменная была рассчитана как сумма всех активностей, умноженных на их веса, и затем делённая на количество просмотров. Таким образом, предсказание конверсии представляет собой задачу регрессии.

3. Модель и эксперименты:
		
		Для работы с изображениями я использовал модель ViT (Vision Transformer), извлёк эмбеддинги изображений и обучил модель CatBoostRegressor для предсказания конверсии.
		Для работы с текстом я использовал модель BERT, извлёк текстовые эмбеддинги и обучил CatBoostRegressor.
		После этого я объединил эмбеддинги изображений и текста и снова обучил CatBoostRegressor на объединённых данных.

4. Метрика и результаты:
		
		В качестве основной метрики я выбрал среднеквадратическую ошибку (MSE). На объединённых данных (изображения и текст) модель достигла MSE = 0.0217, что является хорошим результатом для данной задачи.

In [106]:
import pandas as pd

data = pd.read_csv('post2ctr_dataset.csv')

In [107]:
import torch

print(torch.__version__)
torch.cuda.is_available() 

2.0.0


True

In [3]:
import torch
from PIL import Image
from base64 import b64decode
from io import BytesIO
from transformers import ViTFeatureExtractor, ViTModel

# Инициализация feature extractor и модели ViT
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224')

# Определение устройства (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Перемещаем модель на GPU
vit_model = vit_model.to(device)

c:\Users\eugen\anaconda3\envs\rec_sys\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\eugen\anaconda3\envs\rec_sys\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
def preprocess_image(image_base64):
    # Декодируем изображение
    img = Image.open(BytesIO(b64decode(image_base64))).convert("RGB")
    
    # Преобразуем изображение для подачи в модель
    inputs = feature_extractor(images=img, return_tensors="pt")
    return inputs['pixel_values'].squeeze(0)

In [5]:
def extract_image_features_batch(batch_images):
    # Преобразуем список изображений в батч
    batch_images_tensor = torch.stack(batch_images).to(device)
    
    # Извлекаем признаки с помощью ViT
    with torch.no_grad():
        outputs = vit_model(pixel_values=batch_images_tensor)
    
    # Получаем эмбеддинг изображений (среднее по последнему слою для каждого изображения)
    batch_embeddings = outputs.last_hidden_state.mean(dim=1)
    
    return batch_embeddings

In [6]:
# Предварительная обработка изображений
preprocessed_images = [preprocess_image(img) for img in data['photo']]

In [7]:
# Параметры батчей
batch_size = 32
image_embeddings = []

# Обработка изображений в батчах
for i in range(0, len(preprocessed_images), batch_size):
    batch_images = preprocessed_images[i:i + batch_size]
    batch_embeddings = extract_image_features_batch(batch_images)
    image_embeddings.append(batch_embeddings)

In [8]:
# Объединяем все батчи в один тензор
X = torch.cat(image_embeddings)

In [ ]:
# Переносим эмбеддинги на CPU и создаем датасет из них
X_cpu = X.cpu().numpy()
image_embeddings_df = pd.DataFrame(X_cpu, columns=[f'image_emb_{i}' for i in range(X_cpu.shape[1])])

In [10]:
# Зададим веса для каждой активности
weights = {
    'like': 1.8,           # Лайк 
    'comment': 4,          # Комментарий 
    'hide': 3,             # Скрытие поста 
    'expand': 1.4,         # Развертывание поста 
    'open_photo': 1.3,     # Открытие фото 
    'open': 1.5,           # Открытие поста 
    'share_to_message': 5  # Пересылка в личные сообщения
}

In [12]:
weights_mean = {}
for i in data.columns[1:8]:
	weights_mean[i] = round((1 / data[i].mean()) * 100, 2)

In [80]:
weights_mean

{'like': 0.26,
 'comment': 9.75,
 'hide': 9.31,
 'expand': 0.13,
 'open_photo': 0.11,
 'open': 0.17,
 'share_to_message': 1.79}

In [88]:
# Используется словарь weights
def calculate_weighted_conversion(row):
    weighted_sum = 0
    for activity, weight in weights.items():
        weighted_sum += row[activity] * weight 
        result = weighted_sum / row['view'] if row['view'] > 0 else 0
    return result

data['weighted_conversion'] = data.apply(calculate_weighted_conversion, axis=1)

In [82]:
# Используется словарь weights_mean
def calculate_weighted_conversion(row):
    weighted_sum = 0
    for activity, weight in weights_mean.items():
        weighted_sum += row[activity] * weight * 10 
        result = weighted_sum / row['view'] if row['view'] > 0 else 0
    return result

data['weighted_conversion'] = data.apply(calculate_weighted_conversion, axis=1)

In [89]:
data[['view', 'like', 'comment', 'hide', 'expand', 'open_photo', 'open', 'share_to_message', 'weighted_conversion']].head()

,view,like,comment,hide,expand,open_photo,open,share_to_message,weighted_conversion
0,10869,185,0,2,0,1947,14,20,0.275196
1,9083,227,1,7,4,958,23,2,0.190367
2,5352,25,5,12,598,430,114,4,0.315433
3,4260,539,5,3,1,138,62,24,0.326995
4,5676,112,2,4,371,271,499,4,0.328013


In [90]:
df_image = pd.concat([image_embeddings_df, data['weighted_conversion']], axis=1)

Попробовал применить AutoML для автоматического поиска моделей, но столкнулся с технической ошибкой, которую не удалось решить.

In [100]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


train, test = train_test_split(df_image, test_size=0.2, random_state=42)

task = Task('reg')

automl = TabularAutoML(
		task=task, 
		timeout=3600, 
		cpu_limit=6, 
		gpu_ids='0', 
		# reader_params = {'n_jobs': 6, 'cv': 5, 'random_state': 42, 'verbose': 1},
		general_params= {'use_algos': [['lgb', 'cb', 'nn', 'xgb']]}
)

In [101]:
roles = {
	'target': 'weighted_conversion'
}

In [ ]:
predict = automl.fit_predict(train, roles=roles, verbose=1)
test_pred = automl.predict(test)

In [ ]:
mse = mean_squared_error(y_test, test_pred.data[:, 0])
mse

In [51]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = df_image.drop(columns=['weighted_conversion'])
y = df_image['weighted_conversion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [103]:
model = CatBoostRegressor(iterations=3000, learning_rate=0.1, depth=6, verbose=100)

In [52]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

0:	learn: 0.1711112	total: 180ms	remaining: 6m
100:	learn: 0.1565290	total: 3.82s	remaining: 1m 11s
200:	learn: 0.1444264	total: 7.33s	remaining: 1m 5s
300:	learn: 0.1340942	total: 10.7s	remaining: 1m
400:	learn: 0.1248759	total: 14.1s	remaining: 56.2s
500:	learn: 0.1164777	total: 17.6s	remaining: 52.7s
600:	learn: 0.1090038	total: 21s	remaining: 48.9s
700:	learn: 0.1019725	total: 24.4s	remaining: 45.3s
800:	learn: 0.0954302	total: 27.9s	remaining: 41.7s
900:	learn: 0.0891800	total: 31.3s	remaining: 38.2s
1000:	learn: 0.0835630	total: 34.8s	remaining: 34.7s
1100:	learn: 0.0782189	total: 38.2s	remaining: 31.2s
1200:	learn: 0.0732512	total: 41.6s	remaining: 27.7s
1300:	learn: 0.0685635	total: 45s	remaining: 24.2s
1400:	learn: 0.0643148	total: 48.3s	remaining: 20.7s
1500:	learn: 0.0603371	total: 51.7s	remaining: 17.2s
1600:	learn: 0.0564789	total: 55s	remaining: 13.7s
1700:	learn: 0.0528280	total: 58.4s	remaining: 10.3s
1800:	learn: 0.0494720	total: 1m 1s	remaining: 6.83s
1900:	learn: 0.0

In [53]:
mse = mean_squared_error(y_test, y_pred)
mse

0.02740636255454142

In [54]:
df_image['weighted_conversion'].describe()

count    23527.000000
mean         0.196537
std          0.171080
min          0.000762
25%          0.076025
50%          0.149113
75%          0.265293
max          2.812193
Name: weighted_conversion, dtype: float64

MSE weights: 0.027470408204398132 

MSE weights_mean: 0.11655845174410165 

In [55]:
data['text'] = data['text'].fillna('')

In [56]:
import re

def clean_text(text):
    # Удаляем HTML-теги
    text = re.sub(r'<.*?>', '', text)
    # Заменяем HTML-сущности на пробелы
    text = re.sub(r'&[a-zA-Z0-9#]+;', ' ', text)
    # Разделяем цифры и буквы
    text = re.sub(r'(\d+)([а-яА-Яa-zA-Z])', r'\1 \2', text)  # Цифры перед буквами
    text = re.sub(r'([а-яА-Яa-zA-Z])(\d+)', r'\1 \2', text)  # Буквы перед цифрами
    # Удаляем специальные символы
    text = re.sub(r'[^A-Za-zА-Яа-я0-9ёЁ.,!?;:\s]', '', text)  # Сохраняем буквы, цифры и знаки препинания
    # Удаляем лишние пробелы
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Применяем очистку к колонке 'text'
data['text'] = data['text'].apply(lambda x: clean_text(x) if isinstance(x, str) else x)

In [57]:
data['text']

0                                                         
1                                                         
2        Новость, конечно, старенькая, но все равно инт...
3                                  Фантазийные бриллианты.
4        Сегодня на стадионе Динамо прошли соревнования...
                               ...                        
23522                       Тамара, выиграет в 24 сезоне ?
23523    Продажи Manor Lords превысили 1 млн копий. Сре...
23524                                                     
23525    Магическая фраза: Уже оплачено Позвольте себе ...
23526          Старонемецкая пастушья собака Овечий пудель
Name: text, Length: 23527, dtype: object

In [58]:
from transformers import BertTokenizer, BertModel

# Загружаем предобученный BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

bert_model.to(device)

c:\Users\eugen\anaconda3\envs\rec_sys\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [59]:
def extract_text_features(text):
    # Проверяем, что текст является строкой
    if isinstance(text, str):
        # Токенизация текста
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    else:
        raise ValueError("Input text must be a string.")

    # Извлекаем признаки с помощью BERT
    with torch.no_grad():
        outputs = bert_model(**inputs)

    # Получаем эмбеддинг текста (например, среднее по последнему слою)
    text_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    return text_embedding

In [62]:
text_features = [torch.tensor(extract_text_features(str(txt))) for txt in data['text']]

In [73]:
import numpy as np

text_features_np = np.array(text_features)
text_embeddings_df = pd.DataFrame(text_features_np, columns=[f'text_emb_{i}' for i in range(text_features_np.shape[1])])

In [91]:
df_text = pd.concat([text_embeddings_df, data['weighted_conversion']], axis=1)

In [92]:
X = df_text.drop(columns=['weighted_conversion'])
y = df_text['weighted_conversion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

0:	learn: 0.1681594	total: 41.1ms	remaining: 1m 22s
100:	learn: 0.1399215	total: 3.56s	remaining: 1m 7s
200:	learn: 0.1283066	total: 7.03s	remaining: 1m 2s
300:	learn: 0.1195092	total: 10.4s	remaining: 59s
400:	learn: 0.1118486	total: 13.8s	remaining: 55.1s
500:	learn: 0.1052056	total: 17.2s	remaining: 51.5s
600:	learn: 0.0992370	total: 20.6s	remaining: 48s
700:	learn: 0.0940507	total: 24s	remaining: 44.4s
800:	learn: 0.0890800	total: 27.4s	remaining: 41s
900:	learn: 0.0848134	total: 30.8s	remaining: 37.5s
1000:	learn: 0.0811201	total: 34.2s	remaining: 34.2s
1100:	learn: 0.0775740	total: 37.7s	remaining: 30.7s
1200:	learn: 0.0743590	total: 41.1s	remaining: 27.3s
1300:	learn: 0.0714585	total: 44.5s	remaining: 23.9s
1400:	learn: 0.0688446	total: 47.9s	remaining: 20.5s
1500:	learn: 0.0664760	total: 51.3s	remaining: 17.1s
1600:	learn: 0.0643038	total: 54.8s	remaining: 13.7s
1700:	learn: 0.0624416	total: 58.2s	remaining: 10.2s
1800:	learn: 0.0607378	total: 1m 1s	remaining: 6.81s
1900:	learn

In [93]:
mse = mean_squared_error(y_test, y_pred)
mse

0.02244703374582864

MSE weights: 0.02244703374582864

MSE weights_mean: 0.11835769817232832

In [96]:
df_text_image = pd.concat([df_text.drop(columns='weighted_conversion'), df_image], axis=1)

In [104]:
X = df_text_image.drop(columns=['weighted_conversion'])
y = df_text_image['weighted_conversion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

0:	learn: 0.1682674	total: 73.2ms	remaining: 3m 39s
100:	learn: 0.1375994	total: 7.13s	remaining: 3m 24s
200:	learn: 0.1246470	total: 13.8s	remaining: 3m 11s
300:	learn: 0.1138938	total: 20.4s	remaining: 3m 2s
400:	learn: 0.1052671	total: 26.9s	remaining: 2m 54s
500:	learn: 0.0973995	total: 33.5s	remaining: 2m 47s
600:	learn: 0.0905218	total: 40.2s	remaining: 2m 40s
700:	learn: 0.0842650	total: 46.8s	remaining: 2m 33s
800:	learn: 0.0784789	total: 53.5s	remaining: 2m 26s
900:	learn: 0.0731775	total: 1m	remaining: 2m 19s
1000:	learn: 0.0684116	total: 1m 6s	remaining: 2m 13s
1100:	learn: 0.0638981	total: 1m 13s	remaining: 2m 6s
1200:	learn: 0.0597755	total: 1m 20s	remaining: 2m
1300:	learn: 0.0559463	total: 1m 26s	remaining: 1m 53s
1400:	learn: 0.0521293	total: 1m 33s	remaining: 1m 46s
1500:	learn: 0.0487811	total: 1m 40s	remaining: 1m 40s
1600:	learn: 0.0455337	total: 1m 47s	remaining: 1m 33s
1700:	learn: 0.0425675	total: 1m 53s	remaining: 1m 26s
1800:	learn: 0.0398767	total: 2m	remainin

In [105]:
mse = mean_squared_error(y_test, y_pred)
mse

0.02179526188872042

MSE text + image: 0.02179526188872042